In [ ]:
#@title Mount your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

!pip install transformers datasets evaluate rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 

In [ ]:
import sys
import os
import shutil
import warnings
import pandas as pd
import numpy as np
import json
import torch
import evaluate

from transformers import (
  AutoModelForSeq2SeqLM, 
  DataCollatorForSeq2Seq,
  Seq2SeqTrainingArguments, 
  Seq2SeqTrainer,
  AutoTokenizer, 
  pipeline
) 

from pathlib import Path
from datasets import load_dataset, load_metric

In [ ]:
# Paths

PROJ_DIR = Path('/content/gdrive/MyDrive/IFT6759/quick-recipe')
LOG_DIR = PROJ_DIR / 'logs'

LOG_DIR.mkdir(parents=True, exist_ok=True) 

if str(PROJ_DIR) not in sys.path:
    sys.path.insert(0, str(PROJ_DIR))

MODEL_DIR = PROJ_DIR / 'models'
DATA_DIR = PROJ_DIR / 'youcook2'
MODEL_SAVE_DIR = MODEL_DIR / 'youcook_BART_Coref'
LOG_SAVE_DIR = LOG_DIR / 'youcook_BART_Coref'

MODEL_SAVE_DIR.mkdir(parents=True, exist_ok=True) 
LOG_SAVE_DIR.mkdir(parents=True, exist_ok=True) 

ANNOTATED_DF_PATH = str(DATA_DIR / 'reviewed_0812_coref_aligned.csv')
SPLIT_DF_PATH = str(DATA_DIR / 'train_val_split.csv')
TRAIN_FRAC = 0.7
MAX_INPUT_LENGTH = 1024
MAX_SUMMARY_LENGTH = 128

RANDOM_SEED = 23456
np.random.seed(RANDOM_SEED)

In [ ]:
df = pd.read_csv(ANNOTATED_DF_PATH)

In [ ]:
key_sentences = df[df['IsUsefulSentence'] == 1]

In [ ]:
len(key_sentences)

3569

In [ ]:
split_df = pd.read_csv(SPLIT_DF_PATH)

train_video_urls = list(split_df[split_df['Split'] == 'train']['VideoUrl'].values)
test_video_urls = list(split_df[split_df['Split'] == 'val']['VideoUrl'].values)

train_df = df[df['VideoUrl'].isin(train_video_urls)]
test_df = df[df['VideoUrl'].isin(test_video_urls)]

train_sentences, train_instructions = train_df[train_df['IsUsefulSentence'] == 1]['Sentence_Coref'].to_numpy(), train_df[train_df['IsUsefulSentence'] == 1]['Key steps'].to_numpy()
test_sentences, test_instructions = test_df[test_df['IsUsefulSentence'] == 1]['Sentence_Coref'].to_numpy(), test_df[test_df['IsUsefulSentence'] == 1]['Key steps'].to_numpy()

# Split val set from within train
indices = list(range(len(train_sentences)))
np.random.shuffle(indices)
train_len = int(TRAIN_FRAC * len(train_sentences))

val_sentences, val_instructions = train_sentences[indices[train_len:]], train_instructions[indices[train_len:]]
train_sentences, train_instructions = train_sentences[indices[:train_len]], train_instructions[indices[:train_len]]

In [ ]:
# indices = list(range(len(key_sentences)))
# np.random.shuffle(indices)
# train_len = int(TRAIN_FRAC * len(key_sentences))

In [ ]:
# sentences, instructions = key_sentences['Sentence'].to_numpy() , key_sentences['Key steps'].to_numpy()

In [ ]:
# train_sentences, train_instructions = sentences[:train_len], instructions[:train_len]
# val_sentences, val_instructions = sentences[train_len:], instructions[train_len:]

In [ ]:
len(train_sentences), len(val_sentences), len(train_instructions), len(val_sentences)

(1755, 753, 1755, 753)

In [ ]:
checkpoint = "sshleifer/distilbart-xsum-12-3"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, model_max_length=MAX_INPUT_LENGTH)

In [ ]:
prefix = ""

def preprocess_function(examples, max_input_length=MAX_INPUT_LENGTH, max_summary_length=MAX_SUMMARY_LENGTH):
    inputs = [prefix + doc for doc in examples["text"]]
    
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=max_summary_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [ ]:
tokenizer('hi how are you', max_length=MAX_INPUT_LENGTH, truncation=True)

{'input_ids': [0, 3592, 141, 32, 47, 2], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [ ]:
def generate_encodings(sentences, instructions, tokenizer, max_input_length=MAX_INPUT_LENGTH, max_summary_length=MAX_SUMMARY_LENGTH):
    examples = []
    for sentence, instruction in zip(list(sentences), list(instructions)):
        try:            
            sentence = str(sentence)
            instruction = str(instruction)
            example = {'text': sentence, 'summary': instruction}
            model_inputs = tokenizer(sentence, max_length=max_input_length, truncation=True)
            labels = tokenizer(text_target=instruction, max_length=max_summary_length, truncation=True)
            model_inputs['labels'] = labels['input_ids']
            example['input_ids'] = model_inputs['input_ids']
            example['attention_mask'] = model_inputs['attention_mask']
            example['labels'] = model_inputs['labels']
            examples.append(example)
        except Exception as e:
            print(sentence, instruction)
            continue
    
    return examples

In [ ]:
class YouCookDatasetForKnowledgeExtraction(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        item = self.encodings[idx]
        item['input_ids'] = torch.tensor(item['input_ids'])
        item['attention_mask'] = torch.tensor(item['attention_mask'])
        item['labels'] = torch.tensor(item['labels'])
        return item

    def __len__(self):
        return len(self.encodings)

In [ ]:

train_encodings = generate_encodings(train_sentences, train_instructions, tokenizer)
val_encodings = generate_encodings(val_sentences, val_instructions, tokenizer)
test_encodings = generate_encodings(test_sentences, test_instructions, tokenizer)

train_dataset = YouCookDatasetForKnowledgeExtraction(train_encodings)
val_dataset = YouCookDatasetForKnowledgeExtraction(val_encodings)
test_dataset = YouCookDatasetForKnowledgeExtraction(test_encodings)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
MODEL_SAVE_DIR, LOG_SAVE_DIR

(PosixPath('/content/gdrive/MyDrive/IFT6759/quick-recipe/models/youcook_BART_Coref'),
 PosixPath('/content/gdrive/MyDrive/IFT6759/quick-recipe/logs/youcook_BART_Coref'))

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=str(MODEL_SAVE_DIR),
    logging_dir=str(LOG_SAVE_DIR),
    logging_steps=10,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=3,
    # load_best_model_at_end=False,
    num_train_epochs=6,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.331600,1.976171,0.412800,0.190000,0.398900,0.398200,12.827400
2,1.641600,1.830468,0.430200,0.206400,0.415300,0.414800,12.573700
3,1.598900,1.783984,0.434300,0.209800,0.420500,0.420400,12.761000
4,1.421500,1.774660,0.437300,0.216100,0.421400,0.421800,12.682600
5,1.152300,1.770243,0.436000,0.210900,0.422200,0.422300,12.770300
6,1.197300,1.768727,0.436800,0.213100,0.423600,0.423900,12.776900


<ipython-input-18-b4fc7bd9ae1e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['input_ids'] = torch.tensor(item['input_ids'])
<ipython-input-18-b4fc7bd9ae1e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['attention_mask'] = torch.tensor(item['attention_mask'])
<ipython-input-18-b4fc7bd9ae1e>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(item['labels'])
<ipython-input-18-b4fc7bd9ae1e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or so

TrainOutput(global_step=330, training_loss=1.687768233906139, metrics={'train_runtime': 382.6792, 'train_samples_per_second': 27.517, 'train_steps_per_second': 0.862, 'total_flos': 1022177715763200.0, 'train_loss': 1.687768233906139, 'epoch': 6.0})

## Inference and metrics calculation

In [ ]:
MODEL_CHECKPOINT_PATH = str(MODEL_SAVE_DIR / 'checkpoint-275')

In [ ]:
summarizer = pipeline("summarization", model=MODEL_CHECKPOINT_PATH)

In [ ]:
total_true_pos = 0
total_num_predicted = 0
total_num_gold = 0
text_list = []
summary_list = []
predictions_list = []
true_positive_list = []
num_predicted_list = []
num_gold_list = []

for index in range(len(test_dataset)):
  if (index+1) % 50 == 0:
    print(f"Processing example {index+1}")
  text = test_dataset[index]['text']
  summary = test_dataset[index]['summary']
  text_words = text.split(' ')
  summary_words = summary.split(' ')
  max_len = len(text_words)
  predictions = summarizer(text, min_length=3, max_length=max_len)
  predicted_words = set(predictions[0]['summary_text'].split(' '))
  # print("Text: ", text)
  # print("Predicted: ", predictions)
  # print("Actual: ", summary)
  true_pos = len(set(predicted_words) & set(summary_words))
  num_predicted = len(set(predicted_words))
  num_gold = len(set(summary_words))
  total_true_pos += true_pos
  total_num_predicted += num_predicted
  total_num_gold += num_gold
  text_list.append(text)
  summary_list.append(summary)
  predictions_list.append(predictions[0]['summary_text'])
  true_positive_list.append(true_pos)
  num_predicted_list.append(num_predicted)
  num_gold_list.append(num_gold)

<ipython-input-18-b4fc7bd9ae1e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['input_ids'] = torch.tensor(item['input_ids'])
<ipython-input-18-b4fc7bd9ae1e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['attention_mask'] = torch.tensor(item['attention_mask'])
<ipython-input-18-b4fc7bd9ae1e>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(item['labels'])


Processing example 50
Processing example 100
Processing example 150
Processing example 200
Processing example 250
Processing example 300
Processing example 350
Processing example 400
Processing example 450
Processing example 500
Processing example 550
Processing example 600
Processing example 650
Processing example 700
Processing example 750
Processing example 800
Processing example 850
Processing example 900
Processing example 950
Processing example 1000
Processing example 1050


In [ ]:
predictions_list

['place dough in caputo flour',
 'flip pizza',
 'sprinkle the surface',
 'squish pizza',
 'dimple pizza',
 'Do a quick stretch while rotating',
 'put pizza on the backs of hands',
 'put sauce on pizza',
 'put cheese on',
 'put on olive oil',
 'place pizza on stone',
 'rotrot pizza',
 'put olive oil directly on',
 'cut pizza',
 'stretch dough',
 'add tomato sauce',
 'finish margarita pizza',
 'add tomato sauce',
 'leave pizza',
 'lay bread on bread',
 'lay basil',
 'Press down the mixture',
 'pop dough in oven',
 'keep pizza on pizza stone',
 'cut into bite size pieces',
 'take peanut butter',
 'combine ingredients',
 'put in chili sauce',
 'put in soy sauce',
 'mix salt',
 'heat pan',
 'put in garlic',
 'put ginger in chicken',
 'remove fat',
 'put bread in fan',
 'stir fry mixture',
 'coast',
 'add peanut butter mixture',
 'cook peanut butter',
 'drain water',
 'put tofu on paper towel',
 'take green onion, chopped up carrot',
 'combine egg whites',
 'take green onion',
 'add water',


In [ ]:
_predictions = [prediction for prediction in predictions_list]
predictions_list = _predictions

In [ ]:
results_df = pd.DataFrame({'text': text_list,
                'summary': summary_list,
                'prediction': predictions_list,
                'true_positives': true_positive_list,
                'num_predicted': num_predicted_list,
                'num_gold': num_gold_list
                })

In [ ]:
results_df.to_csv(str(LOG_SAVE_DIR / 'val_performance.csv'), index=False)

In [ ]:
pd.read_csv(str(LOG_SAVE_DIR / 'val_performance.csv'))

,text,summary,prediction,true_positives,num_predicted,num_gold
0,so we ' ve placed the dough directly into the ...,place dough in caputo flour,place dough in caputo flour,5,5,5
1,and then we give the pizza a flip as i ' ve re...,flip dough,flip pizza,1,2,2
2,"but that 's what we do anyway , we sprinkle th...",sprinkle the surface,sprinkle the surface,3,3,3
3,we just give a squish with our palm and make t...,squish dough with palm; flatten center,squish pizza,1,2,6
4,"we dimple the rest of the pizza , moving the p...",dimple the rest of pizza; move the pizza around,dimple pizza,2,2,8
...,...,...,...,...,...,...
1056,"so of vegetable oil into a large , deep and he...",oil skillet,add vegetable oil,1,3,2
1057,"best , if you have one as one holds , and dist...",fry at 365 degrees,heat oil,0,2,4
1058,"the chicken pieces in the skillet and fry , th...",fry for 20 minutes,fry chicken for 15 minutes,3,6,4
1059,"pieces , several times during the cooking time...",drain chicken,drain chicken,2,2,2


In [ ]:
precision = round(total_true_pos / total_num_predicted, 2)
recall = round(total_true_pos / total_num_gold, 2)
f1 = round(2*(precision*recall) / (precision+recall), 2)

print(f"Total true positives (predicted words overlap with gold words): {total_true_pos}")
print(f"Total predicted words: {total_num_predicted}")
print(f"Total gold words: {total_num_gold}")
print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")

Total true positives (predicted words overlap with gold words): 1895
Total predicted words: 3490
Total gold words: 4154
Precision: 0.54, Recall: 0.46, F1: 0.5


## Generate Knowledge Extraction metrics based on best Key-clip Prediction model's predictions

In [ ]:
def best_keyclip_prediction_true_positives(row):
    return row['true_positives'] if row['IsPredUseful'] == 1 else 0

def generate_metrics_from_master_and_val_df(master_df, val_df):
  combined_df = pd.merge(master_df, val_df, left_on='Sentence', right_on='text', how='inner')
  combined_df['best_kc_pred_true_positives'] = combined_df.apply(lambda row: best_keyclip_prediction_true_positives(row), axis=1)
  total_num_gold = combined_df['num_gold'].sum()
  total_num_predicted = combined_df['num_predicted'].sum()
  total_true_pos = combined_df['best_kc_pred_true_positives'].sum()
  precision = round(total_true_pos / total_num_predicted, 2)
  recall = round(total_true_pos / total_num_gold, 2)
  f1 = round(2*(precision*recall) / (precision+recall), 2)

  print(f"Total true positives (predicted words overlap with gold words): {total_true_pos}")
  print(f"Total predicted words: {total_num_predicted}")
  print(f"Total gold words: {total_num_gold}")
  print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
  
  return precision, recall, f1

In [ ]:
master_df = pd.read_pickle(DATA_DIR / 'full_master_updated.pkl')
val_df = pd.read_csv(str(LOG_SAVE_DIR / 'val_performance.csv'))

In [ ]:
generate_metrics_from_master_and_val_df(master_df, val_df)

Total true positives (predicted words overlap with gold words): 532
Total predicted words: 1278
Total gold words: 1544
Precision: 0.42, Recall: 0.34, F1: 0.38


(0.42, 0.34, 0.38)